In [5]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ab-testing/ab_data.csv


In [6]:
dados = pd.read_csv('/kaggle/input/ab-testing/ab_data.csv')
dados.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


De acordo com o e-mail do desafio, as principais colunas do dataset são referentes ao: 
- *ID do usuário*; 
- a *versão do site* ao qual ele foi exposto;
- e se ele converteu (realizou uma compra) ou não.

In [3]:
dados.shape

(294478, 5)

In [4]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


# Distribuição de frequências:

Também há uma variável separando os usuários em dois grupos (Controle e Tratamento), mas sua divisão não corresponde a metade dos usuários.

In [5]:
dados.group.value_counts()

treatment    147276
control      147202
Name: group, dtype: int64

In [6]:
frequencia = dados.group.value_counts()
percentual = dados.group.value_counts(normalize = True) * 100
dist_freq1 = pd.DataFrame({'Frequência': frequencia, 'Porcentagem (%)': percentual})
dist_freq1

,Frequência,Porcentagem (%)
treatment,147276,50.012565
control,147202,49.987435


Ao contrário da versão da página, que está dividida ao meio.

In [7]:
dados.landing_page.value_counts()

old_page    147239
new_page    147239
Name: landing_page, dtype: int64

In [8]:
frequencia = dados.landing_page.value_counts()
percentual = dados.landing_page.value_counts(normalize = True) * 100
dist_freq2 = pd.DataFrame({'Frequência': frequencia, 'Porcentagem (%)': percentual})
dist_freq2

,Frequência,Porcentagem (%)
old_page,147239,50.0
new_page,147239,50.0


In [17]:
dados.user_id.value_counts()

805339    2
754884    2
722274    2
783176    2
898232    2
         ..
642985    1
771499    1
923606    1
712675    1
715931    1
Name: user_id, Length: 290584, dtype: int64

No entanto, ao verificar quantas vezes os IDs dos usuários aparecem, vemos que alguns acessaram a página mais de uma vez e que pode ter causado esta diferença entre as variáveis.

In [9]:
dados.converted.value_counts()

0    259241
1     35237
Name: converted, dtype: int64

In [10]:
frequencia = dados.converted.value_counts()
percentual = dados.converted.value_counts(normalize = True) * 100
dist_freq3 = pd.DataFrame({'Frequência': frequencia, 'Porcentagem (%)': percentual})
dist_freq3

,Frequência,Porcentagem (%)
0,259241,88.034081
1,35237,11.965919


In [11]:
dados.query('landing_page == "old_page"').converted.value_counts()

0    129500
1     17739
Name: converted, dtype: int64

In [12]:
dados.query('landing_page == "new_page"').converted.value_counts()

0    129741
1     17498
Name: converted, dtype: int64

In [13]:
pd.crosstab(dados.converted, dados.landing_page)

landing_page,new_page,old_page
converted,,
0,129741,129500
1,17498,17739


In [14]:
pd.crosstab(dados.converted, dados.landing_page, normalize = True) * 100

landing_page,new_page,old_page
converted,,
0,44.05796,43.97612
1,5.94204,6.02388


De acordo com o *crosstab*, houve uma baixa taxa de conversão e a versão antiga do site performou melhor do que a nova.

# Z test:

Mesmo com esta informação, escolhi executar um *Z-test* para afirmar esta descoberta. 

Utilizei a biblioteca *[statsmodels](https://www.statsmodels.org/dev/generated/statsmodels.stats.proportion.proportions_ztest.html)*, que possui uma função própria para este teste, pois ao informar o número de conversões que ocorreram nos dois grupos e o valor total de cada um, tenho como retorno o teste estatístico para o z-test e o p-valor.

In [16]:
import statsmodels.api as sm

total_nova_pagina = dados.query('landing_page == "new_page"').landing_page.count()
total_antiga_pagina = dados.query('landing_page == "old_page"').landing_page.count()

converted_antigo = dados.query("landing_page == 'old_page' and converted == 1").shape[0]
converted_novo = dados.query("landing_page == 'new_page' and converted == 1").shape[0]

Considerando que: 
- Hipótese nula = a conversão de clientes é igual nas duas páginas (p-valor > 0.05)
- Hipótese alternativa = uma das páginas teve um melhor desempenho (p-valor < 0.05)

In [17]:
z_stat, p_value = sm.stats.proportions_ztest([converted_antigo, converted_novo], [total_antiga_pagina, total_nova_pagina])
z_stat, p_value

(1.3683341399998907, 0.17120750909314209)

Diante este valor, não podemos rejeitar a Hipótese nula.

Portanto, a nova página não é melhor do que a versão antiga.